In [1]:
import pandas as pd

In [2]:
prefix_path = 'D:/jeju_bus_data_no_leakage/bis/'
infix_path = 'BIS_' + '190531'
postfix_path = '.csv'

path = prefix_path + infix_path + postfix_path

In [3]:
df = pd.read_csv(path, 
                 usecols = ['EVENT_DATE', 'EVENT_CD', 'RUN_DIST',
                            'ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT', 'STATION_ORD'])
#df.loc[:, 'EVENT_DATE'] = df['EVENT_DATE'].apply(lambda datetime: pd.to_datetime(datetime))

In [4]:
# df = df[(df['EVENT_CD'] == 18) | (df['EVENT_CD'] == 17)]
df = df.query('EVENT_CD in (17, 18)')

In [5]:
grouped = df.groupby(by = ['ROUTE_ID', 'ROUTE_RUN_CNT', 'VH_ID', 'BUS_RUN_CNT'])

In [6]:
# instances = []

for name, group in grouped:
#     print(name)
    sorted_group = group.sort_values(by = ['EVENT_DATE', 'EVENT_CD'], ascending = [True, True])
    
    # 버스 출발/도착 이벤트 로그가 중복되어 저장된 경우 -> 해당 로그 튜플 삭제
    # 예) EVENT_CD | STATION_ORD (2019.05.31 BIS, 노선ID 405130001, 6회차)
    #     ---------+------------
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1             -> 삭제 대상
    #     17       | 1             -> 삭제 대상
    #     18       | 1
    for station_ord, gr in sorted_group.groupby(by = 'STATION_ORD'):
        if len(gr) > 2:
            #print(name, '...', gr)
            sorted_group = sorted_group.drop(gr.index[:-1])
    
    # 첫 행의 EVENT_CD가 18(출발)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[0]]['EVENT_CD'] == 17:
        sorted_group = sorted_group.drop(sorted_group.index[0])
    
    # 마지막 행의 EVENT_CD가 17(도착)이 아닌 경우 -> 해당 로그 튜플 삭제
    if len(sorted_group) > 0 and \
            sorted_group.loc[sorted_group.index[-1]]['EVENT_CD'] == 18:
        sorted_group = sorted_group.drop(sorted_group.index[-1])
        
    # 출발/도착 결측 값 보정
    tup = (len(sorted_group[sorted_group['EVENT_CD'] == 18]), len(sorted_group[sorted_group['EVENT_CD'] == 17]))
    if tup[0] != tup[1]:
        cnt, start, end = -1, 0, len(sorted_group) - 1
        for station_ord, gr in sorted_group.groupby(by = 'STATION_ORD'):
            cnt += 1
            if cnt in (start, end): 
                continue
            
            if len(gr) == 1:
                if gr.loc[gr.index[0]]['EVENT_CD'] == 18:
                    pass
                else: # 17
                    pass
    
    # {18(출발) - 17(도착)} 테이블 병합
    else: # <- 추후 else 삭제
        cols = ['EVENT_DATE', 'RUN_DIST', 'STATION_ORD']
        start_df = sorted_group.query('EVENT_CD == 18').reset_index()[cols]
        end_df = sorted_group.query('EVENT_CD == 17').reset_index()[cols]

#         for idx, new_col in enumerate(list(pd.Series(cols) + '2')):
#             start_df[new_col] = end_df[cols[idx]]
        for col in cols:
            start_df[col + '2'] = end_df[col]
    
        start_df.loc[:, 'EVENT_DATE'] = start_df['EVENT_DATE'].apply(lambda datetime: pd.to_datetime(datetime))
        start_df['RUN_SPED'] = (start_df['RUN_DIST2'] - start_df['RUN_DIST']) / \
                               (start_df['EVENT_DATE2'] - start_df['EVENT_DATE']) * 3.6
    # 
    
    
    
#         print(name, end = ' ... ')
#         print(tup)
#         instances.append(sorted_group)

TypeError: ufunc subtract cannot use operands with types dtype('<U17') and dtype('<M8[ns]')

In [ ]:
#start_df['RUN_SPED'] = (start_df['RUN_DIST2'] - start_df['RUN_DIST']) / 
#start_df.EVENT_DATE

In [21]:
start_df.EVENT_DATE[0]

Timestamp('2031-05-19 14:24:29')

In [19]:
start_df

,EVENT_DATE,RUN_DIST,STATION_ORD,EVENT_DATE2,RUN_DIST2,STATION_ORD2
0,2031-05-19 14:24:29,1087,3,19/05/31 14:28:40,2207,4
1,2031-05-19 14:29:00,2207,4,19/05/31 14:32:15,4400,5
2,2031-05-19 14:32:41,4400,5,19/05/31 14:35:34,7077,6
3,2031-05-19 14:35:35,7077,6,19/05/31 14:37:48,8101,7
4,2031-05-19 14:38:25,8101,7,19/05/31 14:42:49,9308,8
5,2031-05-19 14:43:51,9308,8,19/05/31 14:45:52,9882,9
6,2031-05-19 14:45:53,9882,9,19/05/31 14:51:48,11483,10
7,2031-05-19 14:51:49,11483,10,19/05/31 14:57:33,13395,11
8,2031-05-19 14:58:27,13395,11,19/05/31 15:00:03,13889,12
9,2031-05-19 15:00:18,13889,12,19/05/31 15:04:33,15122,13
